In [70]:
import pandas as pd

import logging
import os
import re

In [11]:
data_dir = '/home/bartek/Workspace/BigData/data'
metadata_excel = os.path.join(data_dir, 'Kopia Kody_stacji_pomiarowych.xlsx')
measures = {
    'PM10': ['PM10'],
    'PM25': ['PM25', 'PM2.5', 'PM2,5'],
}

In [73]:
def resave(excel_path, csv_path):
    df = pd.read_excel(excel_path, header=None)
    first_col = df.columns[0]
    header = df[df[first_col].str.lower() == 'kod stacji'].values.flatten()
    if len(header) == 0:
        for i, row in df.iterrows():
            if row.str.contains('1g').any():
                header = df.iloc[i-2]
                break
        else:
            logging.warning(f'Cannot parse {excel_path}')
            return
    
    header[0] = 'Date'
    df = df[~pd.to_datetime(df[first_col], errors='coerce').isna()]
    df.columns = header
    df.to_csv(csv_path, index=False)
    
    
def excel_to_csv(dir_path, year, measure):
    m_name, possible_m_names = measure
    for m in possible_m_names:
        excel_name = f'{year}_{m}_1g.xlsx'
        excel_path = os.path.join(dir_path, excel_name)
        if not os.path.exists(excel_path):
            continue
            
        csv_name = f'{m_name}.csv'
        csv_path = os.path.join(dir_path, csv_name)
        resave(excel_path, csv_path)
        return
    logging.warning(f'{m_name} not found in {year}')
    
    
def handle_zip(data_dir, fname):
    if '.zip' not in fname.lower():
        return '', -1
            
    fname = re.sub(r'(.zip)+', ".zip", fname.lower())
    year = fname[:-4]
    if not year.isdecimal():
        return '', -1

    zip_path = os.path.join(data_dir, fname)
    new_dir_path = os.path.join(data_dir, year)
    os.system(f"mkdir {new_dir_path}; unzip {zip_path} -d {new_dir_path}; rm {zip_path};")
    return new_dir_path, year
    
    
def remove_excels(dir_path):
    excels = [os.path.join(dir_path, fname) for fname in os.listdir(dir_path) if '.xlsx' in fname]
    rm_command = 'rm \'' + '\' \''.join(excels) + '\''
    os.system(rm_command)

    
def load_data(data_dir, measures):
    for fname in os.listdir(data_dir):
        new_dir_path, year = handle_zip(data_dir, fname)
        if not new_dir_path:
            continue
            
        for m in measures.items():
            excel_to_csv(new_dir_path, year, m)

        remove_excels(new_dir_path)

In [74]:
load_data(data_dir, measures)

In [11]:
metadata = pd.read_excel(metadata_excel)
metadata.head()

,NR,WOJEWÓDZTWO,KOD STARY,KOD NOWY,NAZWA STACJI,MIEJSCOWOSC,ADRES
0,1,DOLNOŚLĄSKIE,DsBogatMob,DsBogatFrancMOB,Bogatynia Mobil,Bogatynia,ul. Francuska/Kręta
1,2,DOLNOŚLĄSKIE,DsCzer02,DsCzerStraza,Czerniawa,Czerniawa,ul. Strażacka
2,3,DOLNOŚLĄSKIE,DsDzia01,DsDzialoszyn,Działoszyn,Działoszyn,NaN
3,4,DOLNOŚLĄSKIE,DsDzierPilsA,DsDziePilsud,Dzierżoniów - Piłsudskiego,Dzierżoniów,ul. Piłsudskiego
4,5,DOLNOŚLĄSKIE,DsGlogWita,DsGlogWiStwo,Głogów - Wita Stwosza,Głogów,ul. Wita Stwosza


In [13]:
metadata.to_csv('../data/stacje.csv', index=False)